# Stable Mariages

For this workshop we'll implement a version of the **Nobel Prize-winning algorithm** for the stable marriage problem. It's called the [Gale-Shapley](https://en.wikipedia.org/wiki/Gale%E2%80%93Shapley_algorithm) algorithm AKA the "Deferred Acceptance" algorithm.

The Gale Shapley algorithm solves what's called the **Stable Marriage** Problem. Here is the problem framed as marrying men and women: 

> Given $n$ men and $n$ women, where each person has ranked all members of the opposite sex in order of preference, marry the men and women together such that there are no two people of opposite sex who would both rather have each other than their current partners. When there are no such pairs of people, the set of marriages is deemed stable.

This little model however is generally applied to more than marrying men and women. For example, it's the algorithm used to match newly graduated doctors with hospitals, and a variation on it matches kidney patients with organ donors in our hospital system.

The algorithmic question is, given lists of preferences as input, can we find a stable marriage? Can we even guarantee a stable marriage will exist for any set of preferences? The answer to both questions is yes, and it uses an algorithm called deferred acceptance.

Here is an informal description of the algorithm. It goes in rounds. In each round, each man “proposes” to the highest-preferred woman that has not yet rejected him. On the other side, each woman holds a reference to a man at all times. If a woman gets new proposals in a round, she immediately rejects every proposer except her most preferred, but does not accept that proposal. She “defers” the acceptance of the proposal until the very end.

![](Gale-Shapley.gif)

# Your Task

Build a solution `gale_shapley(men, women) -> dict` Where the input is a list of suitors and a list of Suiteds with every one in these lists holding their lists of preferences. The output is a dictionary `suitor -> suited`

Here is a possible way to design a solution:

### Man Class

The `Man(id, preference_list)` class holds the following attributes:

- Its own ID (an `int`)

- A list or array of IDs, which are ordered. So `preference_list[0]` is prefered to `preference_list[1]` and so on

- A method `Suitor.preference()` which points to its current possible reference. It should start by pointing at `preference_list[0]` and every time the Suitor gets rejected in the algorithm, this should point to the next preference.

### Woman Class

The `Woman(id, preference_list)` class holds the following attributes:

- Its own ID (an `int`)

- A list or array of IDs, which are ordered. So `preference_list[0]` is prefered to `preference_list[1]` and so on

- A set of current suitors

- A method `Suited.reject()` which returns all current suitors except the most preferred one

### The Stable Mariage Algorithm

Takes in a list of men and women, loops over suitors trying to find a match until there aren't any unassigned suitors left. Here is the pseudocode for the algorithm:

```
algorithm stable_matching
    Initialize all m ∈ M and w ∈ W to free
    while ∃ free man m who still has a woman w to propose to do
        w := first woman on m's list to whom m has not yet proposed
        if w is free then
            (m, w) become engaged
        else some pair (m', w) already exists
            if w prefers m to m' then
                m' becomes free
                (m, w) become engaged 
            else
                (m', w) remain engaged
            end if
        end if
    repeat
```


Here is some example code of a solution working:

```
men = [
    Man(id=0, preference_list=[0,1,2,3]),
    Man(id=1, preference_list=[2,3,0,1]),
    Man(id=2, preference_list=[1,0,3,2]),
    Man(id=3, preference_list=[3,2,1,0]),
]

women = [
    Woman(id=0, preference_list=[0,1,2,3]),
    Woman(id=1, preference_list=[2,3,0,1]),
    Woman(id=2, preference_list=[1,0,3,2]),
    Woman(id=3, preference_list=[3,2,1,0]),
]

stable_marriage(men, women)
```

output:

```
{0: 0, 
 2: 1, 
 1: 2, 
 3: 3}
```

In [1]:
#Classes

class Man():
    
    def __init__(self, id, preference_list):
        """initialize Man class"""
        self.id = id #index of specific man or suitor in set of Man
        self.preference_list = preference_list #contains the man's ith most preferred woman
        self.rejections = 0 #num rejection is also index of next option
        #The number of rejections the man has seen so far and also
        #the index of the woman that the man is currently proposing to.
    
    def preference(self):
        return self.preference_list[self.rejections]
    
    def __repr__(self):
        return repr(self.id)
    
    
class Woman():
    
    def __init__(self, id, preference_list):
        """initalize Woman class"""
        self.id = id
        self.preference_list = preference_list
        self.held = None #currently held suitor
        self.current_men = set() #list of men currently proposing to woman
    
    def reject(self):
        """
        Accepts no inputs and returns a list
        of rejected suitors while updateing the woman's
        state to hold onto her preferred suitor
        """
        if len(self.current_men) == 0:
            return set()
        if self.held is not None:
            self.current_men.add(self.held)
        
        self.held = min(self.current_men, key=lambda man: #min finds the Man that appears 1st in her pref list
                        self.preference_list.index(man.id))
        rejected = self.current_men - set([self.held])
        self.current_men = set()
        return rejected
        
    def __repr__(self):
        return repr(self.id)

In [2]:
def stable_marriage(men,women):
    """
    Algorith for finding a stable marriage. 
    Follows the deferred acceptance algorigthm.
    """
    unassigned = set(men) #set of unmarried men. All men are unassigned at first.

    #each iteration of loop is a round of the algorithm:
    #The men are added to the current_men list of their next most preferred woman
    while len(unassigned) > 0: #while men are still unmarried
        for man in unassigned:
            women[man.preference()].current_men.add(man)
        unassigned = set()
        
        for woman in women:
            unassigned |= woman.reject() #|= is basically += but for union: unassigned = unassigned | woman.reject() 
            #we combine unassigned and woman reject
        for man in unassigned:
            man.rejections += 1 #Men rejection count increase by one and returned to unassigned men pool
    return dict([(woman.held, woman) for woman in women]) #return a dict of marriaged
#done once all the women are paired with a man

In [3]:
men = [
    Man(id=0, preference_list=[0,1,2,3]),
    Man(id=1, preference_list=[2,3,0,1]),
    Man(id=2, preference_list=[1,0,3,2]),
    Man(id=3, preference_list=[3,2,1,0]),
]

women = [
    Woman(id=0, preference_list=[0,1,2,3]),
    Woman(id=1, preference_list=[2,3,0,1]),
    Woman(id=2, preference_list=[1,0,3,2]),
    Woman(id=3, preference_list=[3,2,1,0]),
]

In [4]:
stable_marriage(men,women)

{0: 0, 2: 1, 1: 2, 3: 3}

In [5]:
men = [
    Man(id=0, preference_list=[0,4,1,2,3]),
    Man(id=1, preference_list=[4,2,3,0,1]),
    Man(id=2, preference_list=[4,1,0,3,2]),
    Man(id=3, preference_list=[4,3,2,1,0]),
    Man(id=4, preference_list=[4,2,3,1,0]),
]

women = [
    Woman(id=0, preference_list=[0,1,4,2,3]),
    Woman(id=1, preference_list=[4,2,3,0,1]),
    Woman(id=2, preference_list=[1,0,3,4,2]),
    Woman(id=3, preference_list=[4,3,2,1,0]),
    Woman(id=4, preference_list=[3,4,2,1,0]),
]

stable_marriage(men, women)

{0: 0, 2: 1, 1: 2, 4: 3, 3: 4}

In [6]:
men = [Man(0, [3,5,4,2,1,0]), Man(1, [2,3,1,0,4,5]),
       Man(2, [5,2,1,0,3,4]), Man(3, [0,1,2,3,4,5]),
       Man(4, [4,5,1,2,0,3]), Man(5, [0,1,2,3,4,5])]

women = [Woman(0, [3,5,4,2,1,0]), Woman(1, [2,3,1,0,4,5]),
         Woman(2, [5,2,1,0,3,4]), Woman(3, [0,1,2,3,4,5]),
         Woman(4, [4,5,1,2,0,3]), Woman(5, [0,1,2,3,4,5])]

stable_marriage(men, women)


{3: 0, 5: 1, 1: 2, 0: 3, 4: 4, 2: 5}